# Chess AI

## Packages

In [1]:
%pip install torch python-chess tqdm numpy

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import numpy as np
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from chess import pgn
from tqdm import tqdm

## Loading Chess Data

In [3]:
def load_pgn(file_path):
    """
    Loads chess games from a PGN file.

    Args:
        file_path (str): Path to the PGN file.  
    Returns:
        list: A list of chess games.
    """
    games = []
    with open(file_path) as f:
        while True:
            game = pgn.read_game(f)
            if game is None:
                break
            games.append(game)
    return games

files = [file for file in os.listdir('data/pgn') if file.endswith('.pgn')]
limit_of_files = min(len(files), 28)
games = []
i = 1
print("Loading files...")
for file in files[:limit_of_files]:
    file_games = load_pgn(os.path.join('data/pgn', file))
    games.extend(file_games)
    if i >= limit_of_files:
        break
    i += 1

print(f"Total games loaded: {len(games)}")

Loading files...
Total games loaded: 41570


In [4]:
from aux_func import create_input_nn, encode_moves

In [5]:
X, y = create_input_nn(games)
X = X[0:25000000]
y = y[0:25000000]
print("Number of samples:", len(y))

Number of samples: 3332761


In [6]:
X = X[0:25000000]
y = y[0:25000000]
y, move_to_int = encode_moves(y)
num_classes = len(move_to_int)

In [7]:
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.long)

C:\Users\lkyza\AppData\Local\Temp\ipykernel_6644\3998235742.py:1: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\torch\csrc\utils\tensor_new.cpp:256.)
  X = torch.tensor(X, dtype=torch.float32)


## Pre Training

In [8]:
from dataset import ChessDataset
from model import ChessModel

In [9]:
# Create Dataset and DataLoader
dataset = ChessDataset(X, y)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

# Check for GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize model, loss function, and optimizer
model = ChessModel(num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

## Training Process

In [ ]:
num_epochs = 50
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in tqdm(dataloader, desc=f"Epoch {epoch+1}/{num_epochs}"):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()

        # gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()

        running_loss += loss.item() 

    epoch_loss = running_loss / len(dataset)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}")

Epoch 1/10: 100%|██████████| 52075/52075 [20:01<00:00, 43.35it/s]  


Epoch 1/10, Loss: 3.0033


Epoch 2/10: 100%|██████████| 52075/52075 [16:50<00:00, 51.55it/s]


Epoch 2/10, Loss: 2.5523


Epoch 3/10: 100%|██████████| 52075/52075 [15:50<00:00, 54.80it/s]


...

Epoch 8/10, Loss: 2.3879


Epoch 9/10: 100%|██████████| 52075/52075 [18:26<00:00, 47.08it/s]  


Epoch 9/10, Loss: 2.3920


Epoch 10/10: 100%|██████████| 52075/52075 [41:58<00:00, 20.67it/s]  

Epoch 10/10, Loss: 2.3995


In [ ]:
# Save the trained model
torch.save(model.state_dict(), './models/50Epoch_chessModel.pth')

import pickle
with open('./models/move_to_int.pkl', 'wb') as f:
    pickle.dump(move_to_int, f)